# Artificial Neural Network

### Importing the libraries

In [3]:
import numpy as np
import pandas as pd
import tensorflow as tf

In [4]:
tf.__version__

'2.4.0'

## Part 1 - Data Preprocessing

### Importing the dataset

In [5]:
dataset = pd.read_csv('Churn_Modelling.csv') # load csv file
X = dataset.iloc[:, 3:-1].values # column 0-2 is irrelavent to the classification, so start from 3 to -1 (just before last column)
y = dataset.iloc[:, -1].values # result, the last column

In [6]:
print(X)

[[619 'France' 'Female' ... 1 1 101348.88]
 [608 'Spain' 'Female' ... 0 1 112542.58]
 [502 'France' 'Female' ... 1 0 113931.57]
 ...
 [709 'France' 'Female' ... 0 1 42085.58]
 [772 'Germany' 'Male' ... 1 0 92888.52]
 [792 'France' 'Female' ... 1 0 38190.78]]


In [7]:
print(y)

[1 0 1 ... 1 1 0]


### Extra Testing

In [24]:
print(X[2]) # example get the row of 3

[502 'France' 0 42 8 159660.8 3 1 0 113931.57]


In [23]:
print(X[:,0]) # example to get first column

[619 608 502 ... 709 772 792]


In [62]:
print(X[:,-1]) # last column

[101348.88 112542.58 113931.57 ... 42085.58 92888.52 38190.78]


### Encoding categorical data

Label Encoding the "Gender" column

need to use LabelEncoder and then fit_transform

In [10]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
X[:, 2] = le.fit_transform(X[:, 2])

In [9]:
print(X)

[[619 'France' 0 ... 1 1 101348.88]
 [608 'Spain' 0 ... 0 1 112542.58]
 [502 'France' 0 ... 1 0 113931.57]
 ...
 [709 'France' 0 ... 0 1 42085.58]
 [772 'Germany' 1 ... 1 0 92888.52]
 [792 'France' 0 ... 1 0 38190.78]]


One Hot Encoding the "Geography" column

### Comment
Label Encoder => no additional column is added
OneHotEncoder => additional columns have been added and use only 0.0 and 1.0 to represent the value,
such as 1.0 0.0 0.0 to represent France, 0.0 0.0 1.0 to represent Spain

pandas is also able to do the OneHotEncoding

data_dum = pd.get_dummies(data)
pd.DataFrame(data_dum)

In [26]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [1])], remainder='passthrough')
X = np.array(ct.fit_transform(X))

In [28]:
print(X)

[[1.0 0.0 0.0 ... 1 1 101348.88]
 [0.0 0.0 1.0 ... 0 1 112542.58]
 [1.0 0.0 0.0 ... 1 0 113931.57]
 ...
 [1.0 0.0 0.0 ... 0 1 42085.58]
 [0.0 1.0 0.0 ... 1 0 92888.52]
 [1.0 0.0 0.0 ... 1 0 38190.78]]


### Splitting the dataset into the Training set and Test set

In [29]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

### Feature Scaling

fit(): Method calculates the parameters μ and σ and saves them as internal objects.

In [30]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [35]:
print(X_train)

[[-1.01460667 -0.5698444   1.74309049 ...  0.64259497 -1.03227043
   1.10643166]
 [-1.01460667  1.75486502 -0.57369368 ...  0.64259497  0.9687384
  -0.74866447]
 [ 0.98560362 -0.5698444  -0.57369368 ...  0.64259497 -1.03227043
   1.48533467]
 ...
 [ 0.98560362 -0.5698444  -0.57369368 ...  0.64259497 -1.03227043
   1.41231994]
 [-1.01460667 -0.5698444   1.74309049 ...  0.64259497  0.9687384
   0.84432121]
 [-1.01460667  1.75486502 -0.57369368 ...  0.64259497 -1.03227043
   0.32472465]]


## Part 2 - Building the ANN

### Initializing the ANN

a sequence of layers and just like a computational graph

In [36]:
ann = tf.keras.models.Sequential()

### Adding the input layer and the first hidden layer

add a fully connectioned layer

In [37]:
ann.add(tf.keras.layers.Dense(units=6, activation='relu'))

### Adding the second hidden layer

In [38]:
ann.add(tf.keras.layers.Dense(units=6, activation='relu'))

### Adding the output layer

In [39]:
ann.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

## Part 3 - Training the ANN

### Compiling the ANN

https://medium.com/%E9%9B%9E%E9%9B%9E%E8%88%87%E5%85%94%E5%85%94%E7%9A%84%E5%B7%A5%E7%A8%8B%E4%B8%96%E7%95%8C/%E6%A9%9F%E5%99%A8%E5%AD%B8%E7%BF%92ml-note-sgd-momentum-adagrad-adam-optimizer-f20568c968db

對於Optimizer來說，learning rate(學習率) η 相當的重要，太小會花費太多時間學習，太大有可能會造成overfitting，無法正確學習，前面幾種Optimizer的學習率 η，都為固定值，而AdaGrad就是會依照梯度去調整 learning rate η 的優化器，Ada對我來說就是Adaptive的意思

Adam Optimizer 其實可以說就是把前面介紹的Momentum 跟 AdaGrad這二種Optimizer做結合，

Adam 保留了 Momentum 對過去梯度的方向做梯度速度調整與Adam對過去梯度的平方值做learning rate的調整，再加上Adam有做參數的”偏離校正”，使得每一次的學習率都會有個確定的範圍，會讓參數的更新較為平穩。

binary_crossentropy交叉熵损失函数，一般用于二分类：

In [40]:
ann.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

### Training the ANN on the Training set

In [41]:
ann.fit(X_train, y_train, batch_size = 32, epochs = 100)

Epoch 1/100
250/250 [==============================] - 1s 891us/step - loss: 0.5913 - accuracy: 0.7203
Epoch 2/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4930 - accuracy: 0.7934: 0s - loss: 0.5092 - accura
Epoch 3/100
250/250 [==============================] - 0s 973us/step - loss: 0.4702 - accuracy: 0.7896
Epoch 4/100
250/250 [==============================] - 0s 855us/step - loss: 0.4495 - accuracy: 0.7972
Epoch 5/100
250/250 [==============================] - 0s 808us/step - loss: 0.4350 - accuracy: 0.8018
Epoch 6/100
250/250 [==============================] - 0s 883us/step - loss: 0.4301 - accuracy: 0.8081
Epoch 7/100
250/250 [==============================] - 0s 904us/step - loss: 0.4141 - accuracy: 0.8113
Epoch 8/100
250/250 [==============================] - 0s 960us/step - loss: 0.4104 - accuracy: 0.8177
Epoch 9/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4051 - accuracy: 0.8209
Epoch 10/100
250/250 [===========================

250/250 [==============================] - 0s 1ms/step - loss: 0.3386 - accuracy: 0.8598
Epoch 81/100
250/250 [==============================] - 0s 862us/step - loss: 0.3393 - accuracy: 0.8590
Epoch 82/100
250/250 [==============================] - 0s 744us/step - loss: 0.3441 - accuracy: 0.8555
Epoch 83/100
250/250 [==============================] - 0s 864us/step - loss: 0.3416 - accuracy: 0.8586
Epoch 84/100
250/250 [==============================] - 0s 843us/step - loss: 0.3360 - accuracy: 0.8667
Epoch 85/100
250/250 [==============================] - 0s 830us/step - loss: 0.3495 - accuracy: 0.8573
Epoch 86/100
250/250 [==============================] - 0s 820us/step - loss: 0.3436 - accuracy: 0.8605
Epoch 87/100
250/250 [==============================] - 0s 790us/step - loss: 0.3351 - accuracy: 0.8632
Epoch 88/100
250/250 [==============================] - 0s 891us/step - loss: 0.3484 - accuracy: 0.8599
Epoch 89/100
250/250 [==============================] - 0s 786us/step - loss: 0

## Part 4 - Making the predictions and evaluating the model

### Predicting the result of a single observation

**Homework**

Use our ANN model to predict if the customer with the following informations will leave the bank: 

Geography: France

Credit Score: 600

Gender: Male

Age: 40 years old

Tenure: 3 years

Balance: \$ 60000

Number of Products: 2

Does this customer have a credit card? Yes

Is this customer an Active Member: Yes

Estimated Salary: \$ 50000

So, should we say goodbye to that customer?

**Solution**

In [42]:
print(ann.predict(sc.transform([[1, 0, 0, 600, 1, 40, 3, 60000, 2, 1, 1, 50000]])) > 0.5)

[[False]]


Therefore, our ANN model predicts that this customer stays in the bank!

**Important note 1:** Notice that the values of the features were all input in a double pair of square brackets. That's because the "predict" method always expects a 2D array as the format of its inputs. And putting our values into a double pair of square brackets makes the input exactly a 2D array.

**Important note 2:** Notice also that the "France" country was not input as a string in the last column but as "1, 0, 0" in the first three columns. That's because of course the predict method expects the one-hot-encoded values of the state, and as we see in the first row of the matrix of features X, "France" was encoded as "1, 0, 0". And be careful to include these values in the first three columns, because the dummy variables are always created in the first columns.

### Predicting the Test set results

### additional testing

reshape =>>> numpy function

import numpy as np

arrayA = np.arange(8)
=>>> arrayA = array([0, 1, 2, 3, 4, 5, 6, 7])

np.reshape(arrayA, (2, 4))
=>>> array([[0, 1, 2, 3],
       [4, 5, 6, 7]])
       
to show the result side by side

### additional testing

In [59]:
np.concatenate((y_test.reshape(len(y_test),1), y_test.reshape(len(y_test),1)),1)

array([[0, 0],
       [1, 1],
       [0, 0],
       ...,
       [0, 0],
       [0, 0],
       [0, 0]], dtype=int64)

In [60]:
y_pred = ann.predict(X_test)
y_pred = (y_pred > 0.5)
print(np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test),1)),1))

[[0 0]
 [0 1]
 [0 0]
 ...
 [0 0]
 [0 0]
 [0 0]]


### Making the Confusion Matrix

In [61]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred) # correct vs prediction
print(cm)
accuracy_score(y_test, y_pred)

[[1510   85]
 [ 197  208]]


0.859